# Predict and Extract Features with Pre-trained Models
This tutorial will work through how to use pre-trained models for predicting and feature extraction.

## Download pre-trained models
A model often contains two parts, the .json file specifying the neural network structure, and the .params file containing the binary parameters. The name convention is name-symbol.json and name-epoch.params, where name is the model name, and epoch is the epoch number.

Here we download a pre-trained Resnet 50-layer model on Imagenet. Other models are available at http://data.mxnet.io/models/

## Jupyter Scala kernel
Add mxnet scala jar which is created as a part of MXNet Scala package installation in classpath as follows:

**Note**: Process to add this jar in your scala kernel classpath can differ according to the scala kernel you are using.

We have used [jupyter-scala kernel](https://github.com/alexarchambault/jupyter-scala) for creating this notebook.

```
classpath.addPath(<path_to_jar>)

e.g
classpath.addPath("mxnet-full_2.11-osx-x86_64-cpu-0.1.2-SNAPSHOT.jar")
```

In [1]:
classpath.addPath("/Users/roshanin/mxnet/scala-package/assembly/osx-x86_64-cpu/target/mxnet-full_2.11-osx-x86_64-cpu-0.1.2-SNAPSHOT.jar")

Import necessary libraries:

In [2]:
import ml.dmlc.mxnet._
import ml.dmlc.mxnet.module.{FitParams, Module}
import scala.collection.immutable.ListMap
import sys.process._

import ml.dmlc.mxnet._
import ml.dmlc.mxnet.module.{FitParams, Module}
import scala.collection.immutable.ListMap
import sys.process._

Download ResNet pretrained model as follows:

In [3]:
"wget http://data.mxnet.io/models/imagenet/resnet/50-layers/resnet-50-symbol.json -P model/"!

"wget http://data.mxnet.io/models/imagenet/resnet/50-layers/resnet-50-0000.params -P model/"!

--2017-03-23 13:06:52--  http://data.mxnet.io/models/imagenet/resnet/50-layers/resnet-50-symbol.json
Resolving data.mxnet.io... 54.208.175.7
Connecting to data.mxnet.io|54.208.175.7|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76172 (74K) [application/json]
Saving to: ‘model/resnet-50-symbol.json’

     0K .......... .......... .......... .......... .......... 67%  134K 0s
    50K .......... .......... ....                            100% 67.4K=0.7s

2017-03-23 13:06:53 (101 KB/s) - ‘model/resnet-50-symbol.json’ saved [76172/76172]

--2017-03-23 13:06:53--  http://data.mxnet.io/models/imagenet/resnet/50-layers/resnet-50-0000.params
Resolving data.mxnet.io... 54.208.175.7
Connecting to data.mxnet.io|54.208.175.7|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102395376 (98M)
Saving to: ‘model/resnet-50-0000.params’

     0K .......... .......... .......... .......... ..........  0%  149K 11m11s
    50K .......... .......... ..........

res2_0: Int = 0
res2_1: Int = 0

## Initialization
We first load the model into memory with loadCheckpoint. It returns the symbol (see [symbol_scala.ipynb](https://github.com/dmlc/mxnet-notebooks/blob/master/scala/basic/symbol_scala.ipynb)) definition of the neural network, and parameters.

In [3]:
//val mod = Module.loadCheckpoint("model/resnet-50", 0)
val (resnet, argParamsResnet, auxParamsResnet) = Model.loadCheckpoint("model/resnet-50", 0)

log4j:WARN No appenders could be found for logger (MXNetJVM).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


resnet: Symbol = ml.dmlc.mxnet.Symbol@301c1c1
argParamsResnet: Map[String, NDArray] = Map(
  "stage1_unit3_conv3_weight" -> ml.dmlc.mxnet.NDArray@89b0a4dc,
  "stage3_unit2_bn1_gamma" -> ml.dmlc.mxnet.NDArray@86c56651,
  "stage2_unit3_conv2_weight" -> ml.dmlc.mxnet.NDArray@a82db0fc,
  "stage4_unit3_bn1_beta" -> ml.dmlc.mxnet.NDArray@7c5beb31,
  "stage1_unit3_bn1_beta" -> ml.dmlc.mxnet.NDArray@a50bbff2,
  "stage3_unit1_conv3_weight" -> ml.dmlc.mxnet.NDArray@cc375d57,
  "stage2_unit1_bn3_gamma" -> ml.dmlc.mxnet.NDArray@a26b6c28,
  "stage2_unit4_conv3_weight" -> ml.dmlc.mxnet.NDArray@bcfcb3fc,
  "stage2_unit4_bn3_gamma" -> ml.dmlc.mxnet.NDArray@c3c51773,
  "stage3_unit4_bn1_gamma" -> ml.dmlc.mxnet.NDArray@49c1a0c1,
  "stage3_unit6_conv2_weight" -> ml.dmlc.mxnet.NDArray@67f8b5ec,
  "stage2_unit2_bn3_gamma" -> ml.dmlc.mxnet.NDArray@f4623b37,
  "stage3_unit2_bn3_beta" -> ml.dmlc.mxnet.NDArray@757458dc,
  "stage1_unit2_bn2_gamma" -> ml.dmlc.mxnet.NDArray@c6a86cd0,
  "stage4_unit1_conv2_weight"

We can visualize the neural network by `Visualization.plotNetwork` and save it by `dot.render` method. Give path where you want to save the visualization as follows: 

In [4]:
val dot = Visualization.plotNetwork(symbol = resnet)
dot.render(engine = "dot", fileName = "resnet", path = "model/")

dot: Visualization.Dot = ml.dmlc.mxnet.Visualization$Dot@3154bd24

Both argument parameters and auxiliary parameters (e.g mean/std in batch normalization layer) are stored as a dictionary of string name and ndarray value (see [ndarray_scala.ipynb](https://github.com/dmlc/mxnet-notebooks/blob/master/scala/basic/ndarray_scala.ipynb). The arguments contain consist of weight and bias.

You can see full output by `println` command

In [5]:
argParamsResnet

res4: Map[String, NDArray] = Map(
  "stage1_unit3_conv3_weight" -> ml.dmlc.mxnet.NDArray@9a4b8176,
  "stage3_unit2_bn1_gamma" -> ml.dmlc.mxnet.NDArray@6abf1f48,
  "stage2_unit3_conv2_weight" -> ml.dmlc.mxnet.NDArray@9df221ba,
  "stage4_unit3_bn1_beta" -> ml.dmlc.mxnet.NDArray@75d224a8,
  "stage1_unit3_bn1_beta" -> ml.dmlc.mxnet.NDArray@640dd0ce,
  "stage3_unit1_conv3_weight" -> ml.dmlc.mxnet.NDArray@abdd2283,
  "stage2_unit1_bn3_gamma" -> ml.dmlc.mxnet.NDArray@b816e9c2,
  "stage2_unit4_conv3_weight" -> ml.dmlc.mxnet.NDArray@7a388895,
  "stage2_unit4_bn3_gamma" -> ml.dmlc.mxnet.NDArray@f6083d12,
  "stage3_unit4_bn1_gamma" -> ml.dmlc.mxnet.NDArray@abfbce39,
  "stage3_unit6_conv2_weight" -> ml.dmlc.mxnet.NDArray@52b0e4a3,
  "stage2_unit2_bn3_gamma" -> ml.dmlc.mxnet.NDArray@fce672e1,
  "stage3_unit2_bn3_beta" -> ml.dmlc.mxnet.NDArray@77fe9b1f,
  "stage1_unit2_bn2_gamma" -> ml.dmlc.mxnet.NDArray@d9e38fd9,
  "stage4_unit1_conv2_weight" -> ml.dmlc.mxnet.NDArray@9ed3362a,
  "stage2_unit2_conv2

while auxiliaries contains the the mean and std for the batch normalization layers.

In [6]:
auxParamsResnet

res5: Map[String, NDArray] = Map(
  "stage2_unit2_bn3_moving_mean" -> ml.dmlc.mxnet.NDArray@aa539dcc,
  "stage3_unit6_bn2_moving_var" -> ml.dmlc.mxnet.NDArray@6e85d8da,
  "stage2_unit2_bn1_moving_mean" -> ml.dmlc.mxnet.NDArray@fdc34046,
  "stage3_unit1_bn2_moving_var" -> ml.dmlc.mxnet.NDArray@8cbb07ea,
  "stage1_unit3_bn1_moving_mean" -> ml.dmlc.mxnet.NDArray@673a1594,
  "stage2_unit4_bn3_moving_mean" -> ml.dmlc.mxnet.NDArray@a2bb7ded,
  "stage2_unit4_bn3_moving_var" -> ml.dmlc.mxnet.NDArray@dacdc026,
  "stage3_unit1_bn3_moving_mean" -> ml.dmlc.mxnet.NDArray@8cbcc0cc,
  "stage2_unit3_bn2_moving_mean" -> ml.dmlc.mxnet.NDArray@fe5da542,
  "stage2_unit1_bn3_moving_var" -> ml.dmlc.mxnet.NDArray@9a7dfe4b,
  "stage1_unit2_bn1_moving_var" -> ml.dmlc.mxnet.NDArray@698017c1,
  "stage4_unit3_bn1_moving_mean" -> ml.dmlc.mxnet.NDArray@6ea56feb,
  "stage1_unit2_bn1_moving_mean" -> ml.dmlc.mxnet.NDArray@b32bb14c,
  "stage3_unit6_bn3_moving_mean" -> ml.dmlc.mxnet.NDArray@733075d0,
  "stage3_unit5_bn2

Next we create an executable module (see [module_scala.ipynb](https://github.com/dmlc/mxnet-notebooks/blob/master/scala/basic/module_scala.ipynb)) on GPU 0. To use a difference device, we just need to change the context, e.g. Context.cpu(0) for CPU and Context.gpu(2) for the 3rd GPU.

In [6]:
val mod = new Module(resnet, contexts = Context.cpu())

mod: Module = ml.dmlc.mxnet.module.Module@56ca816d

The ResNet is trained with RGB images of size 224 x 224. The training data is feed by the variable data. We bind the module with the input shape and specify that it is only for predicting. The number 1 added before the image shape (3x224x224) means that we will only predict one image each time. Next we set the loaded parameters. Now the module is ready to run.

In [5]:
val dataShapesResnet = ListMap("data" -> Shape(1, 3, 224, 224))
mod.bind(dataShapes=dataShapesResnet, forTraining = false)
//mod.setParams(argParams, auxParams)

: 

In [16]:
mod.dataShapes
mod.dataNames
//argParams

res15_0: IndexedSeq[DataDesc] = Vector(DataDesc[data,(1,3,224,224),Float32,NCHW])
res15_1: IndexedSeq[String] = Vector("data")

In [17]:
mod.setParams(argParamsResnet, auxParamsResnet)

: 